In [1]:
import os
import matplotlib.pyplot as plt
import datetime
import time
import matplotlib.gridspec as gridspec

import pickle
import numpy as np
import pandas as pd
import seaborn as sns

from itertools import compress

from helpers.expr_data import ExprData
from helpers.scale_data import ScaleData
from helpers.similarity import Similarity
from helpers.feature_selection import FeatureSelection

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import root_mean_squared_error as rmse_score
from sklearn.metrics import make_scorer

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
SMALL_SMALL_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
# plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize

plt.rc('legend', fontsize=SMALL_SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
OVERALL_PLOT = True

In [4]:
X_label = 'SKU'
expr_label = 'EXPR'
y_true_label = 'Y_TRUE'
y_pred_label = 'Y_PRED'
suffix_labels = ['_small', '_large']

In [5]:
import random
np.random.seed(47907)
random.seed(47907)

In [6]:
def nrmse_score(y_true, y_pred):
    # return np.sqrt((((y_true-y_pred)/y_pred)**2).mean())
    # return (abs(y_true-y_pred)/y_pred).mean()
    # return rmse_score(y_true, y_pred)/(np.mean(y_true))
    return rmse_score(y_true, y_pred)/(np.max(y_true)-np.min(y_true))
score_func = make_scorer(nrmse_score, greater_is_better=False)

In [7]:
# read in the performance metrics for each experiment
data_all = ExprData()
data_all.load_pickle()
data = data_all.remove_by_wlname(['xml', 'ycsb'])
ycsb_data = data_all.remove_by_wlname(['xml', 'tpcc', 'tpch', 'twitter'])

In [8]:
data = data.fix_tpch()
data = data.merge_tpch()

In [9]:
# similarity for all
new_data = data.keep_complete_exprs()

scaler = ScaleData()
plan_mtxs, plan_col_ranges = scaler.scale(new_data.plan_mtxs)
perf_mtxs, perf_col_ranges = scaler.scale(new_data.perf_mtxs)

simi_calc_all = Similarity(new_data, plan_mtxs, plan_col_ranges, perf_mtxs, perf_col_ranges)
simi_calc_all.calc_bined_mtx() # all plan features

simi_calc = simi_calc_all
fs = FeatureSelection(simi_calc)

In [10]:
sampled_data = new_data.sample_data()

In [11]:
data_by_type = sampled_data.split_by_type()

In [12]:
sampled_by_type = sampled_data.split_by_type()
for ty, expr_set in sampled_by_type.items():
    sub_by_term = expr_set.split_by_term()
    sampled_by_type[ty] = sub_by_term

In [13]:
def get_cpu_nums_as_X(l):
    return np.array([int(e[3:]) for e in l]).reshape(-1, 1)

In [14]:
all_results = {}
all_times = {}

In [20]:
X_label = 'cpu_num'
y_label = 'latency'

In [25]:
def get_score_by_cpus(X, y, cpu_a, cpu_b):
    bigger_mask = [x_lab == cpu_b for x_lab in X.flatten()]
    smaller_mask = [x_lab == cpu_a for x_lab in X.flatten()]

    curr_y_true = y[bigger_mask]
    # curr_y_pred = y[smaller_mask]*(cpu_b/cpu_a)
    curr_y_pred = y[smaller_mask]*(cpu_a/cpu_b)
    n_rmse = nrmse_score(curr_y_true, curr_y_pred)
    return n_rmse

def get_baseline_scores(X, y, wl_name, grouping_type, groupping_id):  
    train_rmses, test_rmses = [], []
    k = 5
    num_cpus = np.sort(np.unique(X))
    train_time = 0

    kf = StratifiedKFold(n_splits=k, shuffle=True)
    for train_index, test_index in kf.split(X, X.flatten()):
        start = time.time()

        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]

        end = time.time()
        train_time += end - start
        for i in range(len(num_cpus)):
            for j in range(i, len(num_cpus)):
                if i == j:
                    continue
                cpu_a = num_cpus[i] # smaller
                cpu_b = num_cpus[j] # larger

                test_rmses.append(get_score_by_cpus(X_test, y_test, cpu_a, cpu_b))
                train_rmses.append(get_score_by_cpus(X_train, y_train, cpu_a, cpu_b))

    
    train_time /= k
    overall_test_rmse = np.mean(test_rmses)
    overall_train_rmse = np.mean(train_rmses)
    return overall_test_rmse, overall_train_rmse, train_time

In [26]:
all_tests, all_trains = [], []
all_results['Baseline'] = {}
all_times['Baseline'] = {}

for ty, curr_data in data_by_type.items():
    name = curr_data.wl_names[0]
    term = curr_data.terminal_num[0]
    if name not in all_results['Baseline']:
        all_results['Baseline'][name] = {}
        all_times['Baseline'][name] = {}
    print(f'Group Id: {ty}, wl name {name}, terminal num {term}')
    
    X = get_cpu_nums_as_X(curr_data.cpu_nums)
    y = np.array(curr_data.wl_latency)
    # y = np.array(curr_data.wl_throughput)
    test_r2_mean, train_r2_mean, train_time = get_baseline_scores(X, y, curr_data.wl_names[0], 'group', 
                                                                     curr_data.wl_groups[0])
    print('Test rmse = {}, Train rmse = {}'.format(test_r2_mean, train_r2_mean))
    all_tests.append(test_r2_mean)
    all_trains.append(train_r2_mean)
    all_results['Baseline'][name][term] = test_r2_mean
    all_times['Baseline'][name][term] = train_time
print('Overall test nrmse: {}, train {}'.format(np.mean(all_tests), np.mean(all_trains)))

Group Id: 1, wl name tpcc, terminal num 32
Test rmse = 17.958665969123523, Train rmse = 11.687422938024717
Group Id: 2, wl name tpch, terminal num 1
Test rmse = 0.5515308448026249, Train rmse = 0.44111751462776394
Group Id: 3, wl name twitter, terminal num 32
Test rmse = 90.96997590140539, Train rmse = 63.784158884038305
Group Id: 4, wl name twitter, terminal num 8
Test rmse = 67.34117237789143, Train rmse = 43.33022795519159
Group Id: 7, wl name tpcc, terminal num 8
Test rmse = 12.503082486878801, Train rmse = 8.876188073772932
Group Id: 8, wl name tpcc, terminal num 4
Test rmse = 9.896702360513567, Train rmse = 8.202993399119906
Group Id: 9, wl name twitter, terminal num 4
Test rmse = 21.071702537854375, Train rmse = 16.40488132784905
Overall test nrmse: 31.470404639781385, train 21.81814144180347


In [27]:
terminals = [4, 8, 32]
workloads = ['tpcc', 'twitter', 'tpch']
methods = ['Baseline']
for me in methods:
    curr_line = ''
    scs = []
    times = []
    for wl in workloads:
        if wl == 'tpch':
            sc = abs(all_results[me][wl][1])
            curr_line += f' {sc:.3f} &'
            scs.append(sc)
            times.append(all_times[me][wl][1])
        else:
            for ter in terminals:
                sc = abs(all_results[me][wl][ter])
                curr_line += f' {sc:.3f} &'
                scs.append(sc)
                times.append(all_times[me][wl][ter])
    curr_line += f' {np.mean(scs):.3}'
    curr_line += " \\\\ \cline{2-11}"
    curr_line = f'& {me} & ' + f' {np.mean(times):.4f} & ' + curr_line
    print(curr_line)

& Baseline &  0.0000 &  9.897 & 12.503 & 17.959 & 21.072 & 67.341 & 90.970 & 0.552 & 31.5 \\ \cline{2-11}


In [29]:
terminals = [4, 8, 32]
workloads = ['tpcc', 'twitter', 'tpch']
methods = ['Baseline']

for me in methods:
    curr_line = ''
    scs = []
    times = []
    for wl in workloads:
        if wl == 'tpch':
            sc = abs(all_results[me][wl][1])
            curr_line += ' \hlrfive{'
            curr_line += f'{sc:.3f}'
            curr_line += '} &'
            scs.append(sc)
            times.append(all_times[me][wl][1])
        else:
            for ter in terminals:
                sc = abs(all_results[me][wl][ter])
                curr_line += ' \hlrfive{'
                curr_line += f'{sc:.3f}'
                curr_line += '} &'
                scs.append(sc)
                times.append(all_times[me][wl][ter])
    curr_line += '\hlrfive{' + f'{np.mean(scs):.3f}'+'}'
    # curr_line += " \\\\ \cline{2-11}"
    curr_line += " \\\\ \hline"
    curr_line = '&\hlrfive{' + f'{me}' + '} & \hlrfive{' + f'{np.mean(times):.4f}' + '} & ' + curr_line
    print(curr_line)

&\hlrfive{Baseline} & \hlrfive{0.0000} &  \hlrfive{9.897} & \hlrfive{12.503} & \hlrfive{17.959} & \hlrfive{21.072} & \hlrfive{67.341} & \hlrfive{90.970} & \hlrfive{0.552} &\hlrfive{31.470} \\ \hline
